In [1]:
from contextualized_topic_models.models.kitty_classifier import Kitty
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk
import torch
import random
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from nltk.corpus import stopwords
!jupyter nbextension enable --py widgetsnbextension

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# nltk.download("stopwords")

/home/ippk93/anaconda3/envs/made_1/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
from contextualized_tm.subset import get_data_sample

In [3]:
# torch.manual_seed(10)
# torch.cuda.manual_seed(10)
# np.random.seed(10)
# random.seed(10)
# torch.backends.cudnn.enabled = False
# torch.backends.cudnn.deterministic = True

In [4]:
spark = SparkSession.builder.appName("Kitty").getOrCreate()

22/10/23 02:56:54 WARN Utils: Your hostname, PK resolves to a loopback address: 127.0.1.1; using 172.28.78.27 instead (on interface eth0)
22/10/23 02:56:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/23 02:56:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
DATA_PATH = '../../../notebooks/MADE/data.parquet'  # Input path to dataset in parquet format

In [6]:
ids, abstracts = get_data_sample(spark=spark, data_path=DATA_PATH, sample_frac=0.01)

In [7]:
list(zip(ids[:5], abstracts[:5]))

[('556fa3b02401b4b38c237522',
  'An N:1 protection mechanism is proposed to reduce backup OLTs in a resilient dual-homed LR-PON deployment. We model the problem as an Integer Linear Program and solve it for an Irish network deployment.'),
 ('556fa12b2401b4b38c237465',
  'Multi-tenant data centers provide a cost-effective many-server infrastructure for hosting large-scale applications. These data centers can run multiple virtual machines (VMs) for each tenant, and potentially place any of these VMs on any of the servers. Therefore, for inter-VM communication, they also need to provide a VM resolution method that can quickly determine the server location of any VM. Unfortunately, existing methods suffer from a scalability bottleneck in the network load of the address resolution messages and/or in the size of the resolution tables. In this paper, we propose Smart Address Learning (SAL), a novel approach that expands the scalability of both the network load and the resolution table sizes, 

In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
kt = Kitty()
kt.train(abstracts, topics=10, embedding_model="allenai/scibert_scivocab_uncased", stopwords_list=list(stop_words), epochs=5)

No sentence-transformers model found with name /home/ippk93/.cache/torch/sentence_transformers/allenai_scibert_scivocab_uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/ippk93/.cache/torch/sentence_transformers/allenai_scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly 

Batches:   0%|          | 0/148 [00:00<?, ?it/s]

Epoch: [5/5]	 Seen Samples: [147570/147570]	Train Loss: 454.0544805007793	Time: 0:00:03.230704: : 5it [00:16,  3.20s/it] 
Sampling: [20/20]: : 20it [00:37,  1.87s/it]


In [12]:
print(*[(i, kt.get_word_classes(10)[i]) for i in range(10)], sep='\n')

(0, ['data', 'learning', 'based', 'classification', 'text', 'set', 'results', 'method', 'approach', 'mining'])
(1, ['performance', 'network', 'transmission', 'power', 'channel', 'energy', 'proposed', 'wireless', 'scheme', 'interference'])
(2, ['software', 'development', 'process', 'systems', 'information', 'tools', 'design', 'research', 'knowledge', 'use'])
(3, ['image', 'method', 'images', 'using', 'based', 'proposed', 'segmentation', 'resolution', 'algorithm', 'recognition'])
(4, ['problem', 'control', 'model', 'algorithm', 'numerical', 'time', 'linear', 'nonlinear', 'problems', 'optimization'])
(5, ['services', 'data', 'network', 'service', 'applications', 'cloud', 'systems', 'security', 'distributed', 'application'])
(6, ['conditional', 'parallelism', 'commercial', 'page', 'operate', 'additionally', 'back', 'commonly', 'quasi', 'latter'])
(7, ['robot', 'human', 'students', 'learning', 'interaction', 'subjects', 'study', 'using', 'user', 'social'])
(8, ['networking', 'advances', 'me

In [14]:
kt.assigned_classes = {
    0 : "machine learning",
    1 : "Distributed systems",
    2 : "Software development",
    3 : "Computer vision",
    4 : "Signal processing",
    5 : "Cloud & Web",
    6 : "other",
    7 : "Motion systems",
    8 : "Networks",
    9 : "Graphs"
}

In [15]:
text = """
Dynamic Voltage and Frequency Scaling (DVFS) is commonly used to save energy in computing systems.
However, when it comes to parallel programs, existing DVFS controllers only reduce frequency while or before waiting in blocking communications. As a consequence, energy savings are only possible for the program tasks out of the critical path and when the workload is imbalanced.
We propose a new runtime DVFS controller, FoREST-mn.
It allows to take advantage of both the low CPU usage of some program phases as well as communication slack to save more energy with parallel programs.
The DVFS control then becomes more complex, but energy savings are even obtained when the workload is balanced.
The resulting slowdown on programs is carefully controlled and constrained by a user-defined threshold.
We implemented the presented strategies and evaluated it on 4 compute nodes totaling 64 cores.
FoREST-mn is able to perform significant CPU energy savings on the NAS programs, up to 34 % on MG, while efficiently bounding the resulting slowdown.
"""

kt.predict([text])

No sentence-transformers model found with name /home/ippk93/.cache/torch/sentence_transformers/allenai_scibert_scivocab_uncased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/ippk93/.cache/torch/sentence_transformers/allenai_scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Sampling: [20/20]: : 20it [00:10,  1.89it/s]


['Distributed systems']

### See results of trained (on another sample) model

#### Результаты ниже могут не работать, это нормально

In [20]:
parquet = spark.read.parquet('contextualized_tm/preds.parquet')

In [6]:
df = parquet.toPandas()

In [25]:
df.head()

,old_id,theme
0,556fa6752401b4b38c2375ea,Robotics
1,556faa7e2401b4b38c2376ef,other
2,556fab9a2401b4b38c23773c,Algorithms
3,556fabc12401b4b38c237744,Signal Processing
4,556fabc52401b4b38c237748,Computer Vision


In [18]:
df['theme'].value_counts()

other                                           9658
Computer Vision                                 6676
Project Development                             6042
Worldwide Web                                   5839
Graph Theory                                    5587
Software Verification                           5481
Algorithms                                      5407
Robotics                                        5380
Natural Language Processing                     5339
Recommendation Systems                          5219
Learning Courses                                5090
Computer Medicine                               5085
Machine Learning                                5044
Optimization Methods                            4966
Software Engineering                            4919
Signal Processing                               4845
Formal Systems                                  4788
Databases                                       4739
Devices                                       

In [17]:
len(df[df['theme'] != 'other']) / len(df['theme'])

0.9338882576016867